<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inference-example-with-a-Cellpose-model:-www.cellpose.org" data-toc-modified-id="Inference-example-with-a-Cellpose-model:-www.cellpose.org-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inference example with a Cellpose model: <a href="http://www.cellpose.org" rel="nofollow" target="_blank">www.cellpose.org</a></a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#I'm-running-it-in-a-separate-process-rather-than-as-a-regular-notebook-because-I've-faced-issues-with-numpy-version-not-updating-after-the-pip-install-above.-If-you-know-how-to-fix-this-please-let-me-know." data-toc-modified-id="I'm-running-it-in-a-separate-process-rather-than-as-a-regular-notebook-because-I've-faced-issues-with-numpy-version-not-updating-after-the-pip-install-above.-If-you-know-how-to-fix-this-please-let-me-know.-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>I'm running it in a separate process rather than as a regular notebook because I've faced issues with numpy version not updating after the <code>pip install</code> above. If you know how to fix this please let me know.</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Inference example with a Cellpose model: www.cellpose.org
The model is based on U-Net, however rather than training it directly on bitmask targets they first convert them to "spatial flows" representations and train on that. This makes segmentation of dense and touching cells more reliable. For details and additional tricks they use see the paper "Cellpose: a generalist algorithm for cellular segmentation".

To train it I used the script provided in the cellpose repo ie: `python -m cellpose --train ...` after I converted the dataset to the input format it expects.

In inference I just submit the masks as they were returned from the model - no postprocessing

In [1]:
!pip install --no-index ../input/cellpose-library/offline_whls/cellpose-0.7.2-py3-none-any.whl --find-links=../input/cellpose-library/offline_whls

Looking in links: ../input/cellpose-library/offline_whls
Processing /home/cozy/Documents/projects/kaggle/input/cellpose-library/offline_whls/cellpose-0.7.2-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/cozy/Documents/projects/kaggle/input/cellpose-library/offline_whls/cellpose-0.7.2-py3-none-any.whl'



#### I'm running it in a separate process rather than as a regular notebook because I've faced issues with numpy version not updating after the `pip install` above. If you know how to fix this please let me know.

In [2]:
%%writefile run.py
import numpy as np
from cellpose import models, io, plot
from pathlib import Path
import pandas as pd

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

test_dir = Path('dataset/test')
test_files = [fname for fname in test_dir.iterdir()]
model = models.CellposeModel(gpu=True, pretrained_model='models/cellpose_residual_on_style_on_concatenation_off_train_converted_2021_12_17_05_56_19.790502')

ids, masks = [],[]
for fn in test_files:
    preds, flows, _ = model.eval(io.imread(str(fn)), diameter=19, channels=[0,0], augment=True, resample=True)
    for i in range (1, preds.max() + 1):
        ids.append(fn.stem)
        masks.append(rle_encode(preds == i))
        
pd.DataFrame({'id':ids, 'predicted':masks}).to_csv('submission.csv', index=False)

Overwriting run.py


In [3]:
!python run.py

Traceback (most recent call last):
  File "/home/cozy/Documents/projects/kaggle/sartorius-cell-instance-segmentation/run.py", line 2, in <module>
    from cellpose import models, io, plot
ImportError: cannot import name 'models' from 'cellpose' (unknown location)


In [4]:
import pandas as pd
pd.read_csv('submission.csv').head()

ModuleNotFoundError: No module named 'pandas'